# コンテキストエンジニアリング - OpenAI Agents SDKのセッションを使用した短期記憶管理

AIエージェントは多くの場合、**長時間にわたる複数ターンのやり取り**で動作し、適切な**コンテキスト**のバランスを保つことが重要です。引き継ぐ情報が多すぎると、モデルは注意散漫、非効率、または完全な失敗のリスクを負います。保持する情報が少なすぎると、エージェントは一貫性を失います。

ここで、コンテキストとは、モデルが一度に注意を向けることができるトークンの総ウィンドウ（入力 + 出力）を指します。[GPT-5](https://platform.openai.com/docs/models/gpt-5)の場合、この容量は最大272kの入力トークンと128kの出力トークンですが、このような大きなウィンドウでも、整理されていない履歴、冗長なツール結果、またはノイズの多い検索結果によって圧迫される可能性があります。これにより、コンテキスト管理は最適化だけでなく、必要不可欠なものとなります。

このクックブックでは、**[OpenAI Agents SDK](https://github.com/openai/openai-agents-python)の`Session`オブジェクトを使用してコンテキストを効果的に管理する方法**を探求し、エージェントを高速、信頼性が高く、コスト効率的に保つための2つの実証済みのコンテキスト管理技術—**トリミング**と**圧縮**—に焦点を当てます。

#### コンテキスト管理が重要な理由

* **長いスレッド全体での持続的な一貫性** – 古い詳細を引きずることなく、エージェントを最新のユーザー目標に固定し続けます。セッションレベルのトリミングと要約により、「昨日の計画」が今日の要求を上書きすることを防ぎます。
* **より高いツール呼び出し精度** – 焦点を絞ったコンテキストにより、関数の選択と引数の入力が改善され、マルチツール実行中の再試行、タイムアウト、連鎖的な失敗が減少します。
* **低レイテンシ & コスト削減** – より小さく、より鋭いプロンプトにより、ターンあたりのトークンと注意負荷が削減されます。
* **エラー & 幻覚の封じ込め** – 要約は以前の間違いを修正または省略する「クリーンルーム」として機能し、トリミングにより悪い事実（「コンテキスト汚染」）をターンごとに増幅することを回避します。
* **より簡単なデバッグ & 可観測性** – 安定した要約と境界のある履歴により、ログが比較可能になります：要約を差分比較し、回帰を特定し、失敗を確実に再現できます。
* **マルチ問題とハンドオフの耐性** – マルチ問題チャットでは、問題ごとのミニ要約により、エージェントは一貫性を保ちながら一時停止/再開、人間へのエスカレーション、または他のエージェントへのハンドオフが可能になります。

![AI エージェントにおけるメモリ比較](../../images/memory_comparison.jpg)

[OpenAI Responses API](https://platform.openai.com/docs/api-reference/responses/create#responses-create-previous_response_id)は、組み込みの状態と`previous_response_id`によるメッセージチェーンを通じて**基本的なメモリサポート**を提供しています。

前回のレスポンスの`id`を`previous_response_id`として渡すことで会話を継続できます。または、出力をリストに収集し、次のレスポンスの`input`として再送信することで、手動でコンテキストを管理することもできます。

しかし、**自動メモリ管理**は提供されません。そこで**Agents SDK**の出番です。このSDKはResponsesの上に[セッションメモリ](https://openai.github.io/openai-agents-python/sessions/)を提供するため、手動で`response.output`を追加したり、IDを自分で追跡したりする必要がなくなります。セッションが**メモリオブジェクト**となり、単純に`session.run("...")`を繰り返し呼び出すだけで、SDKがコンテキスト長、履歴、継続性を処理してくれます。これにより、一貫性のあるマルチターンエージェントの構築がはるかに簡単になります。

#### 実際のシナリオ

これらの技術を実践的な例で説明します。長時間実行されるタスクの一般的な例として、以下のようなものがあります：

* **マルチターンカスタマーサービス会話**
ハードウェアとソフトウェアの両方を含む技術製品に関する長時間の会話では、顧客は時間の経過とともに複数の問題を提起することがよくあります。エージェントは、過去のすべての詳細を引きずるのではなく、必要最小限の情報のみを保持しながら、一貫性を保ち、目標に集中し続ける必要があります。

#### 取り上げる技術

これらの課題に対処するため、OpenAI Agents SDKを使用した2つの具体的なアプローチを紹介します：

- **コンテキストトリミング** – 古いターンを削除し、最新のNターンのみを保持する。
  - **メリット**

    * **決定論的でシンプル：** 要約器の変動がなく、状態の推論や実行の再現が容易。
    * **レイテンシの追加なし：** 履歴を圧縮するための追加のモデル呼び出しが不要。
    * **最近の作業の忠実性：** 最新のツール結果、パラメータ、エッジケースが逐語的に保持される—デバッグに最適。
    * **「要約ドリフト」のリスクが低い：** 事実を再解釈や圧縮することがない。

    **デメリット**

    * **長期コンテキストの急激な忘却：** 重要な初期の制約、ID、決定がNを超えてスクロールすると消失する可能性。
    * **ユーザーエクスペリエンスの「健忘症」：** エージェントが長いセッションの途中で約束や以前の設定を「忘れた」ように見える。
    * **シグナルの無駄：** 古いターンには再利用可能な知識（要件、制約）が含まれている可能性があるが、それが削除される。
    * **トークンスパイクの可能性：** 最近のターンに巨大なツールペイロードが含まれている場合、最新Nターンでもコンテキストが爆発する可能性。

  - **最適な使用場面**

    - 会話内のタスクが互いに独立しており、重複しないコンテキストで、以前の詳細を引き継ぐ必要がない場合。
    - 予測可能性、簡単な評価、低レイテンシが必要な場合（運用自動化、CRM/APIアクション）。
    - 会話の有用なコンテキストがローカルな場合（最近のステップが遠い履歴よりもはるかに重要）。

- **コンテキスト要約** – 以前のメッセージ（アシスタント、ユーザー、ツールなど）を構造化された短い要約に圧縮し、会話履歴に注入する。

  - **メリット**

    * **長期記憶をコンパクトに保持：** 過去の要件、決定、根拠がNを超えて持続。
    * **よりスムーズなUX：** エージェントが長いセッション全体で約束や制約を「記憶」。
    * **コスト制御されたスケール：** 1つの簡潔な要約で数百のターンを置き換え可能。
    * **検索可能なアンカー：** 単一の合成アシスタントメッセージが安定した「これまでの世界の状態」となる。

    **デメリット**

    * **要約の損失とバイアス：** 詳細が削除されたり、重み付けが間違ったりする可能性；微妙な制約が消失する可能性。
    * **レイテンシとコストのスパイク：** 各更新でモデル作業（および潜在的にツールトリムロジック）が追加。
    * **エラーの複合：** 悪い事実が要約に入ると、将来の動作を**汚染**する可能性（「コンテキスト汚染」）。
    * **観測可能性の複雑さ：** 監査可能性と評価のために要約プロンプト/出力をログに記録する必要。

  - **最適な使用場面**

    - 計画/コーチング、RAG重要分析、ポリシーQ&Aなど、フロー全体で収集されたコンテキストが必要なユースケースがある場合。
    - 長期間にわたる継続性が必要で、関連タスクを解決するために重要な詳細を引き継ぐ必要がある場合。
    - セッションがNターンを超えるが、決定、ID、制約を確実に保持する必要がある場合。
<br>

**簡単な比較**

| 次元         | **トリミング（最新N回の会話）**         | **要約（古い会話 → 生成された要約）** |
| ----------------- | ------------------------------- | ------------------------------------ |
| レイテンシ / コスト    | 最低（追加の呼び出しなし）     | 要約更新時により高い |
| 長期記憶    | 弱い（ハードカットオフ）         | 強い（コンパクトな引き継ぎ）   |
| リスクタイプ         | コンテキストの損失                | コンテキストの歪み/汚染     |
| 可観測性     | シンプルなログ                 | 要約プロンプト/出力のログが必要 |
| 評価の安定性    | 高い                        | 堅牢な要約評価が必要       |
| 最適な用途          | ツール重視の操作、短いワークフロー | アナリスト/コンシェルジュ、長いスレッド      |

## 前提条件

このクックブックを実行する前に、以下のアカウントを設定し、いくつかのセットアップ作業を完了する必要があります。これらの前提条件は、このプロジェクトで使用されるAPIと連携するために必要不可欠です。

#### ステップ0: OpenAIアカウントと`OPENAI_API_KEY`

- **目的:**  
  このクックブックで紹介されている言語モデルにアクセスし、Agents SDKを使用するためにOpenAIアカウントが必要です。

- **手順:**  
  まだアカウントをお持ちでない場合は、[OpenAIアカウントにサインアップ](https://openai.com)してください。アカウントを取得したら、[OpenAI API Keysページ](https://platform.openai.com/api-keys)にアクセスしてAPIキーを作成してください。

**ワークフローを実行する前に、環境変数を設定してください：**

```
# Your openai key
os.environ["OPENAI_API_KEY"] = "sk-proj-..."
```

または、agentsライブラリをインポートして`set_default_openai_key`関数を使用することで、エージェントが使用するOpenAI APIキーを設定することもできます。

```
from agents import set_default_openai_key
set_default_openai_key("YOUR_API_KEY")
```

#### ステップ1: 必要なライブラリのインストール

以下では、`openai-agents`ライブラリ（[OpenAI Agents SDK](https://github.com/openai/openai-agents-python)）をインストールします。

In [ ]:
%pip install openai-agents nest_asyncio

In [1]:
from openai import OpenAI

client = OpenAI()

In [2]:
from agents import set_tracing_disabled
set_tracing_disabled(True)

インストールしたライブラリをテストするために、エージェントを定義して実行してみましょう。

In [ ]:
import asyncio
from agents import Agent, Runner


agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
)

result = await Runner.run(agent, "Tell me why it is important to evaluate AI agents.")
print(result.final_output)


Evaluating AI agents ensures reliability, safety, ethical alignment, performance accuracy, and helps avoid biases, improving overall trust and effectiveness.


### エージェントの定義

Agents SDK ライブラリから必要なコンポーネントを定義することから始めることができます。エージェント作成時にユースケースに基づいて指示が追加されます。

#### カスタマーサービス担当者

In [214]:
support_agent = Agent(
    name="Customer Support Assistant",
    model="gpt-5",
    instructions=(
        "You are a patient, step-by-step IT support assistant. "
        "Your role is to help customers troubleshoot and resolve issues with devices and software. "
        "Guidelines:\n"
        "- Be concise and use numbered steps where possible.\n"
        "- Ask only one focused, clarifying question at a time before suggesting next actions.\n"
        "- Track and remember multiple issues across the conversation; update your understanding as new problems emerge.\n"
        "- When a problem is resolved, briefly confirm closure before moving to the next.\n"
    )
)


## コンテキストトリミング

#### カスタムセッションオブジェクトの実装

[OpenAI Agents Python SDK](https://openai.github.io/openai-agents-python/)の[Session](https://openai.github.io/openai-agents-python/sessions/)オブジェクトを使用しています。以下は**最後のN回のターンのみを保持する**`TrimmingSession`の実装です（「ターン」とは、1つのユーザーメッセージと次のユーザーメッセージまでのすべて—アシスタントの返答やツール呼び出し/結果を含む）。これはインメモリで動作し、書き込みと読み込みのたびに自動的にトリミングを行います。

In [252]:
from __future__ import annotations

import asyncio
from collections import deque
from typing import Any, Deque, Dict, List, cast

from agents.memory.session import SessionABC
from agents.items import TResponseInputItem  # dict-like item

ROLE_USER = "user"


def _is_user_msg(item: TResponseInputItem) -> bool:
    """Return True if the item represents a user message."""
    # Common dict-shaped messages
    if isinstance(item, dict):
        role = item.get("role")
        if role is not None:
            return role == ROLE_USER
        # Some SDKs: {"type": "message", "role": "..."}
        if item.get("type") == "message":
            return item.get("role") == ROLE_USER
    # Fallback: objects with a .role attr
    return getattr(item, "role", None) == ROLE_USER


class TrimmingSession(SessionABC):
    """
    Keep only the last N *user turns* in memory.

    A turn = a user message and all subsequent items (assistant/tool calls/results)
    up to (but not including) the next user message.
    """

    def __init__(self, session_id: str, max_turns: int = 8):
        self.session_id = session_id
        self.max_turns = max(1, int(max_turns))
        self._items: Deque[TResponseInputItem] = deque()  # chronological log
        self._lock = asyncio.Lock()

    # ---- SessionABC API ----

    async def get_items(self, limit: int | None = None) -> List[TResponseInputItem]:
        """Return history trimmed to the last N user turns (optionally limited to most-recent `limit` items)."""
        async with self._lock:
            trimmed = self._trim_to_last_turns(list(self._items))
            return trimmed[-limit:] if (limit is not None and limit >= 0) else trimmed

    async def add_items(self, items: List[TResponseInputItem]) -> None:
        """Append new items, then trim to last N user turns."""
        if not items:
            return
        async with self._lock:
            self._items.extend(items)
            trimmed = self._trim_to_last_turns(list(self._items))
            self._items.clear()
            self._items.extend(trimmed)

    async def pop_item(self) -> TResponseInputItem | None:
        """Remove and return the most recent item (post-trim)."""
        async with self._lock:
            return self._items.pop() if self._items else None

    async def clear_session(self) -> None:
        """Remove all items for this session."""
        async with self._lock:
            self._items.clear()

    # ---- Helpers ----

    def _trim_to_last_turns(self, items: List[TResponseInputItem]) -> List[TResponseInputItem]:
        """
        Keep only the suffix containing the last `max_turns` user messages and everything after
        the earliest of those user messages.

        If there are fewer than `max_turns` user messages (or none), keep all items.
        """
        if not items:
            return items

        count = 0
        start_idx = 0  # default: keep all if we never reach max_turns

        # Walk backward; when we hit the Nth user message, mark its index.
        for i in range(len(items) - 1, -1, -1):
            if _is_user_msg(items[i]):
                count += 1
                if count == self.max_turns:
                    start_idx = i
                    break

        return items[start_idx:]

    # ---- Optional convenience API ----

    async def set_max_turns(self, max_turns: int) -> None:
        async with self._lock:
            self.max_turns = max(1, int(max_turns))
            trimmed = self._trim_to_last_turns(list(self._items))
            self._items.clear()
            self._items.extend(trimmed)

    async def raw_items(self) -> List[TResponseInputItem]:
        """Return the untrimmed in-memory log (for debugging)."""
        async with self._lock:
            return list(self._items)


以下のように、`max_turns=3`で実装したカスタムセッションオブジェクトを定義しましょう。

In [280]:
# Keep only the last 8 turns (user + assistant/tool interactions)
session = TrimmingSession("my_session", max_turns=3)

**適切な`max_turns`の選び方**

このパラメータを決定するには、通常、会話履歴を使った実験が必要です。一つのアプローチは、会話全体のターン数の合計を抽出し、その分布を分析することです。もう一つの選択肢は、LLMを使って会話を評価することです。各会話に含まれるタスクや問題の数を特定し、問題あたりに必要なターン数の平均を計算します。

In [281]:
message = "There is a red light blinking on my laptop."

In [282]:
result = await Runner.run(
    support_agent,
    message,
    session=session
)

In [283]:
history = await session.get_items()


In [284]:
history

[{'content': 'There is a red light blinking on my laptop.', 'role': 'user'},
 {'id': 'rs_68be66229c008190aa4b3c5501f397080fdfa41323fb39cb',
  'summary': [],
  'type': 'reasoning',
  'content': []},
 {'id': 'msg_68be662f704c8190969bdf539701a3e90fdfa41323fb39cb',
  'content': [{'annotations': [],
    'text': 'A blinking red light usually indicates a power/battery or hardware fault, but the meaning varies by brand.\n\nWhat is the exact make and model of your laptop?\n\nWhile you check that, please try these quick checks:\n1) Note exactly where the red LED is (charging port, power button, keyboard edge) and the blink pattern (e.g., constant blink, 2 short/1 long).\n2) Plug the charger directly into a known‑good wall outlet (no power strip), ensure the charger tip is fully seated, and look for damage to the cable/port. See if the LED behavior changes.\n3) Leave it on charge for 30 minutes in case the battery is critically low.\n4) Power reset: unplug the charger; if the battery is removable

In [285]:
# Example flow
await session.add_items([{"role": "user", "content": "I am using a macbook pro and it has some overheating issues too."}])
await session.add_items([{"role": "assistant", "content": "I see. Let's check your firmware version."}])
await session.add_items([{"role": "user", "content": "Firmware v1.0.3; still failing."}])
await session.add_items([{"role": "assistant", "content": "Could you please try a factory reset?"}])
await session.add_items([{"role": "user", "content": "Reset done; error 42 now."}])
await session.add_items([{"role": "assistant", "content": "Leave it on charge for 30 minutes in case the battery is critically low. Is there any other error message?"}])
await session.add_items([{"role": "user", "content": "Yes, I see error 404 now."}])
await session.add_items([{"role": "assistant", "content": "Do you see it on the browser while accessing a website?"}])
# At this point, with max_turns=3, everything *before* the earliest of the last 3 user
# messages is summarized into a synthetic pair, and the last 3 turns remain verbatim.

history = await session.get_items()
# Pass `history` into your agent runner / responses call as the conversation context.


In [286]:
len(history)

6

In [287]:
history

[{'role': 'user', 'content': 'Firmware v1.0.3; still failing.'},
 {'role': 'assistant', 'content': 'Could you please try a factory reset?'},
 {'role': 'user', 'content': 'Reset done; error 42 now.'},
 {'role': 'assistant',
  'content': 'Leave it on charge for 30 minutes in case the battery is critically low. Is there any other error message?'},
 {'role': 'user', 'content': 'Yes, I see error 404 now.'},
 {'role': 'assistant',
  'content': 'Do you see it on the browser while accessing a website?'}]

以下では、`max_turns=3`でのトリミングセッションの動作を確認できます。

![セッションでのコンテキストトリミング](../../images/trimingSession.jpg)

**「ターン」とは何か**

* **ターン** = 1つの**ユーザー**メッセージ**とそれに続くすべて**（アシスタントの返信、推論、ツール呼び出し、ツール結果）が**次のユーザーメッセージまで**続く単位。

**トリミングが発生するタイミング**

* **書き込み時**: `add_items(...)`が新しいアイテムを追加し、その後すぐに保存された履歴をトリミングします。
* **読み込み時**: `get_items(...)`は**トリミングされた**ビューを返します（書き込みをバイパスしても、読み込み時に古いターンが漏れることはありません）。

**保持するものを決定する方法**

1. `role == "user"`を持つアイテムを**ユーザーメッセージ**として扱います（`_is_user_msg`経由）。
2. 履歴を**後ろから**スキャンし、最後の**N**個のユーザーメッセージ（`max_turns`）のインデックスを収集します。
3. それらN個のユーザーメッセージの中で**最も早い**インデックスを見つけます。
4. **そのインデックスから最後まですべてを保持**し、それより前のものはすべて削除します。

これにより、各完全なターン境界が保持されます：保持される最も早いユーザーメッセージがインデックス`k`にある場合、`k`の後に来たすべてのアシスタント/ツールアイテムも保持されます。

**簡単な例**

履歴（古い → 新しい）：

```
0: user("Hi")
1: assistant("Hello!")
2: tool_call("lookup")
3: tool_result("…")
4: user("It didn't work")
5: assistant("Try rebooting")
6: user("Rebooted, now error 42")
7: assistant("On it")
```

`max_turns = 2`の場合、最後の2つのユーザーメッセージはインデックス**4**と**6**にあります。
それらの中で最も早いのは**4** → アイテム**4..7**を保持し、**0..3**を削除します。

**これがうまく機能する理由**

* 常に**完全な**ターンを保持するため、アシスタントは必要な直近のコンテキスト（ユーザーの最後の質問とその間のアシスタント/ツールステップの両方）を保持できます。
* 単なるメッセージではなく、古いターンを丸ごと破棄することで、コンテキストの肥大化を防ぎます。

**カスタマイズのオプション**

* 初期化時に`max_turns`を変更。
* アイテムスキーマが異なる場合は`_is_user_msg(...)`を調整。
* **メッセージ数**や**トークン数**で制限したい場合は、`_trim_to_last_turns(...)`を置き換えるか、トークンを測定する2番目のパスを追加。

## コンテキスト要約

履歴が`max_turns`を超えると、最新のNユーザーターンをそのまま保持し、**それより古いものをすべて2つの合成メッセージに要約します**：

* `user`: *"これまでの会話を要約してください。"*
* `assistant`: *{生成された要約}*

ユーザーからの要約リクエストのシャドウプロンプトは、ユーザーとアシスタント間のチャットフローを混乱させることなく、会話の自然な流れを保つために追加されます。生成された要約の最終版は、アシスタントメッセージに注入されます。

**要約プロンプト**

適切に作成された要約プロンプトは、会話のコンテキストを保持するために不可欠であり、常に特定のユースケースに合わせて調整する必要があります。これは**カスタマーサポート担当者が次の担当者にケースを引き継ぐ**ような状況と考えてください。スムーズに継続するために、どのような簡潔でありながら重要な詳細情報が必要でしょうか？プロンプトは適切なバランスを取る必要があります：不要な情報で過負荷にならず、かつ重要なコンテキストが失われるほど簡素でもない状態です。このバランスを実現するには、詳細レベルを微調整するための慎重な設計と継続的な実験が必要です。

In [66]:
SUMMARY_PROMPT = """
You are a senior customer-support assistant for tech devices, setup, and software issues.
Compress the earlier conversation into a precise, reusable snapshot for future turns.

Before you write (do this silently):
- Contradiction check: compare user claims with system instructions and tool definitions/logs; note any conflicts or reversals.
- Temporal ordering: sort key events by time; the most recent update wins. If timestamps exist, keep them.
- Hallucination control: if any fact is uncertain/not stated, mark it as UNVERIFIED rather than guessing.

Write a structured, factual summary ≤ 200 words using the sections below (use the exact headings):

• Product & Environment:
  - Device/model, OS/app versions, network/context if mentioned.

• Reported Issue:
  - Single-sentence problem statement (latest state).

• Steps Tried & Results:
  - Chronological bullets (include tool calls + outcomes, errors, codes).

• Identifiers:
  - Ticket #, device serial/model, account/email (only if provided).

• Timeline Milestones:
  - Key events with timestamps or relative order (e.g., 10:32 install → 10:41 error).

• Tool Performance Insights:
  - What tool calls worked/failed and why (if evident).

• Current Status & Blockers:
  - What’s resolved vs pending; explicit blockers preventing progress.

• Next Recommended Step:
  - One concrete action (or two alternatives) aligned with policies/tools.

Rules:
- Be concise, no fluff; use short bullets, verbs first.
- Do not invent new facts; quote error strings/codes exactly when available.
- If previous info was superseded, note “Superseded:” and omit details unless critical.
"""


**メモリ要約プロンプト設計の主要原則**

* **マイルストーン:** 会話における重要なイベントを強調する—例えば、問題が解決された時、価値のある情報が発見された時、または必要な詳細がすべて収集された時など。

* **ユースケース特化:** 圧縮プロンプトを特定のユースケースに合わせて調整する。同じタスクを解決する際に、人間がワーキングメモリで情報をどのように追跡し思い出すかを考える。

* **矛盾チェック:** 要約が自己矛盾せず、システム指示やツール定義と競合しないことを確認する。これは推論モデルにとって特に重要で、コンテキストにおける競合が起こりやすい。

* **タイムスタンプと時間的流れ:** 要約にイベントのタイミングを組み込む。これにより、モデルが順序立てて更新について推論でき、タイムライン上で最新のメモリを忘れたり思い出したりする際の混乱を減らす。

* **チャンク化:** 詳細を長い段落ではなく、カテゴリやセクションに整理する。構造化されたグループ化により、LLMが情報の断片間の関係を理解する能力が向上する。

* **ツールパフォーマンスの洞察:** マルチターンでツールを活用したインタラクションから得られた教訓を捉える—例えば、特定のクエリに対してどのツールが効果的に機能したか、そしてその理由を記録する。これらの洞察は将来のステップを導く上で価値がある。

* **ガイダンスと例:** 明確なガイダンスで要約を導く。可能な場合は、会話履歴から具体的な例を抽出して、将来のターンをより根拠に基づいた、コンテキストに富んだものにする。

* **幻覚制御:** 含める内容について正確性を保つ。要約における軽微な幻覚でさえ、前方に伝播し、将来のコンテキストを不正確性で汚染する可能性がある。

* **モデル選択:** ユースケース要件、要約の長さ、レイテンシとコストのトレードオフに基づいて要約モデルを選択する。場合によっては、AIエージェント自体と同じモデルを使用することが有利になることもある。

In [289]:
class LLMSummarizer:
    def __init__(self, client, model="gpt-4o", max_tokens=400, tool_trim_limit=600):
        self.client = client
        self.model = model
        self.max_tokens = max_tokens
        self.tool_trim_limit = tool_trim_limit

    async def summarize(self, messages: List[Item]) -> Tuple[str, str]:
        """
        Create a compact summary from `messages`.

        Returns:
            Tuple[str, str]: The shadow user line to keep dialog natural,
            and the model-generated summary text.
        """
        user_shadow = "Summarize the conversation we had so far."
        TOOL_ROLES = {"tool", "tool_result"}

        def to_snippet(m: Item) -> str | None:
            role = (m.get("role") or "assistant").lower()
            content = (m.get("content") or "").strip()
            if not content:
                return None
            # Trim verbose tool outputs to keep prompt compact    
            if role in TOOL_ROLES and len(content) > self.tool_trim_limit:
                content = content[: self.tool_trim_limit] + " …"
            return f"{role.upper()}: {content}"

        # Build compact, trimmed history
        history_snippets = [s for m in messages if (s := to_snippet(m))]

        prompt_messages = [
            {"role": "system", "content": SUMMARY_PROMPT},
            {"role": "user", "content": "\n".join(history_snippets)},
        ]

        resp = await asyncio.to_thread(
            self.client.responses.create,
            model=self.model,
            input=prompt_messages,
            max_output_tokens=self.max_tokens,
        )

        summary = resp.output_text
        await asyncio.sleep(0)  # yield control
        return user_shadow, summary


In [296]:
import asyncio
from collections import deque
from typing import Optional, List, Tuple, Dict, Any

Record = Dict[str, Dict[str, Any]]  # {"msg": {...}, "meta": {...}}

class SummarizingSession:
    """
    Session that keeps only the last N *user turns* verbatim and summarizes the rest.

    - A *turn* starts at a real user message and includes everything until the next real user message.
    - When the number of real user turns exceeds `context_limit`, everything before the earliest
      of the last `keep_last_n_turns` user-turn starts is summarized into a synthetic user→assistant pair.
    - Stores full records (message + metadata). Exposes:
        • get_items():           model-safe messages only (no metadata)
        • get_full_history():    [{"message": msg, "metadata": meta}, ...]
    """

    # Only these keys are ever sent to the model; the rest live in metadata.
    _ALLOWED_MSG_KEYS = {"role", "content", "name"}

    def __init__(
        self,
        keep_last_n_turns: int = 3,
        context_limit: int = 3,
        summarizer: Optional["Summarizer"] = None,
        session_id: Optional[str] = None,
    ):
        assert context_limit >= 1
        assert keep_last_n_turns >= 0
        assert keep_last_n_turns <= context_limit, "keep_last_n_turns should not be greater than context_limit"

        self.keep_last_n_turns = keep_last_n_turns
        self.context_limit = context_limit
        self.summarizer = summarizer
        self.session_id = session_id or "default"

        self._records: deque[Record] = deque()
        self._lock = asyncio.Lock()

    # --------- public API used by your runner ---------
    async def get_items(self, limit: Optional[int] = None) -> List[Dict[str, Any]]:
        """Return model-safe messages only (no metadata)."""
        async with self._lock:
            data = list(self._records)
        msgs = [self._sanitize_for_model(rec["msg"]) for rec in data]
        return msgs[-limit:] if limit else msgs

    async def add_items(self, items: List[Dict[str, Any]]) -> None:
        """Append new items and, if needed, summarize older turns."""
        # 1) Ingest items
        async with self._lock:
            for it in items:
                msg, meta = self._split_msg_and_meta(it)
                self._records.append({"msg": msg, "meta": meta})

            need_summary, boundary = self._summarize_decision_locked()

        # 2) No summarization needed → just normalize flags and exit
        if not need_summary:
            async with self._lock:
                self._normalize_synthetic_flags_locked()
            return

        # 3) Prepare summary prefix (model-safe copy) outside the lock
        async with self._lock:
            snapshot = list(self._records)
            prefix_msgs = [r["msg"] for r in snapshot[:boundary]]

        user_shadow, assistant_summary = await self._summarize(prefix_msgs)

        # 4) Re-check and apply summary atomically
        async with self._lock:
            still_need, new_boundary = self._summarize_decision_locked()
            if not still_need:
                self._normalize_synthetic_flags_locked()
                return

            snapshot = list(self._records)
            suffix = snapshot[new_boundary:]  # keep-last-N turns live here

            # Replace with: synthetic pair + suffix
            self._records.clear()
            self._records.extend([
                {
                    "msg": {"role": "user", "content": user_shadow},
                    "meta": {
                        "synthetic": True,
                        "kind": "history_summary_prompt",
                        "summary_for_turns": f"< all before idx {new_boundary} >",
                    },
                },
                {
                    "msg": {"role": "assistant", "content": assistant_summary},
                    "meta": {
                        "synthetic": True,
                        "kind": "history_summary",
                        "summary_for_turns": f"< all before idx {new_boundary} >",
                    },
                },
            ])
            self._records.extend(suffix)

            # Ensure all real user/assistant messages explicitly have synthetic=False
            self._normalize_synthetic_flags_locked()

    async def pop_item(self) -> Optional[Dict[str, Any]]:
        """Pop the latest message (model-safe), if any."""
        async with self._lock:
            if not self._records:
                return None
            rec = self._records.pop()
            return dict(rec["msg"])

    async def clear_session(self) -> None:
        """Remove all records."""
        async with self._lock:
            self._records.clear()

    def set_max_turns(self, n: int) -> None:
        """
        Back-compat shim for old callers: update `context_limit`
        and clamp `keep_last_n_turns` if needed.
        """
        assert n >= 1
        self.context_limit = n
        if self.keep_last_n_turns > self.context_limit:
            self.keep_last_n_turns = self.context_limit

    # Full history (debugging/analytics/observability)

    async def get_full_history(self, limit: Optional[int] = None) -> List[Dict[str, Any]]:
        """
        Return combined history entries in the shape:
          {"message": {role, content[, name]}, "metadata": {...}}
        This is NOT sent to the model; for logs/UI/debugging only.
        """
        async with self._lock:
            data = list(self._records)
        out = [{"message": dict(rec["msg"]), "metadata": dict(rec["meta"])} for rec in data]
        return out[-limit:] if limit else out

    # Back-compat alias
    async def get_items_with_metadata(self, limit: Optional[int] = None) -> List[Dict[str, Any]]:
        return await self.get_full_history(limit)

    # Internals

    def _split_msg_and_meta(self, it: Dict[str, Any]) -> Tuple[Dict[str, Any], Dict[str, Any]]:
        """
        Split input into (msg, meta):
          - msg keeps only _ALLOWED_MSG_KEYS; if role/content missing, default them.
          - everything else goes under meta (including nested "metadata" if provided).
          - default synthetic=False for real user/assistant unless explicitly set.
        """
        msg = {k: v for k, v in it.items() if k in self._ALLOWED_MSG_KEYS}
        extra = {k: v for k, v in it.items() if k not in self._ALLOWED_MSG_KEYS}
        meta = dict(extra.pop("metadata", {}))
        meta.update(extra)

        msg.setdefault("role", "user")
        msg.setdefault("content", str(it))

        role = msg.get("role")
        if role in ("user", "assistant") and "synthetic" not in meta:
            meta["synthetic"] = False
        return msg, meta

    @staticmethod
    def _sanitize_for_model(msg: Dict[str, Any]) -> Dict[str, Any]:
        """Drop anything not allowed in model calls."""
        return {k: v for k, v in msg.items() if k in SummarizingSession._ALLOWED_MSG_KEYS}

    @staticmethod
    def _is_real_user_turn_start(rec: Record) -> bool:
        """True if record starts a *real* user turn (role=='user' and not synthetic)."""
        return (
            rec["msg"].get("role") == "user"
            and not rec["meta"].get("synthetic", False)
        )

    def _summarize_decision_locked(self) -> Tuple[bool, int]:
        """
        Decide whether to summarize and compute the boundary index.

        Returns:
            (need_summary, boundary_idx)

        If need_summary:
          • boundary_idx is the earliest index among the last `keep_last_n_turns`
            *real* user-turn starts.
          • Everything before boundary_idx becomes the summary prefix.
        """
        user_starts: List[int] = [
            i for i, rec in enumerate(self._records) if self._is_real_user_turn_start(rec)
        ]
        real_turns = len(user_starts)

        # Not over the limit → nothing to do
        if real_turns <= self.context_limit:
            return False, -1

        # Keep zero turns verbatim → summarize everything
        if self.keep_last_n_turns == 0:
            return True, len(self._records)

        # Otherwise, keep the last N turns; summarize everything before the earliest of those
        if len(user_starts) < self.keep_last_n_turns:
            return False, -1  # defensive (shouldn't happen given the earlier check)

        boundary = user_starts[-self.keep_last_n_turns]

        # If there is nothing before boundary, there is nothing to summarize
        if boundary <= 0:
            return False, -1

        return True, boundary

    def _normalize_synthetic_flags_locked(self) -> None:
        """Ensure all real user/assistant records explicitly carry synthetic=False."""
        for rec in self._records:
            role = rec["msg"].get("role")
            if role in ("user", "assistant") and "synthetic" not in rec["meta"]:
                rec["meta"]["synthetic"] = False

    async def _summarize(self, prefix_msgs: List[Dict[str, Any]]) -> Tuple[str, str]:
        """
        Ask the configured summarizer to compress the given prefix.
        Uses model-safe messages only. If no summarizer is configured,
        returns a graceful fallback.
        """
        if not self.summarizer:
            return ("Summarize the conversation we had so far.", "Summary unavailable.")
        clean_prefix = [self._sanitize_for_model(m) for m in prefix_msgs]
        return await self.summarizer.summarize(clean_prefix)


![セッション内のコンテキストトリミング](../../images/summarizingSession.jpg)

**高レベルのアイデア**

* **1ターン** = 1つの**実際のユーザー**メッセージ**とそれに続くすべて**（アシスタントの返答、ツール呼び出し/結果など）**次の実際のユーザーメッセージまで**。
* 2つのパラメータを設定します：

  * **`context_limit`**: 要約を行う前に生履歴で許可される**実際のユーザーターン**の最大数。
  * **`keep_last_n_turns`**: 要約を行う際に、最新の**ターン**のうち何個をそのまま保持するか。

    * 不変条件: `keep_last_n_turns <= context_limit`。
* **実際の**ユーザーターン数が`context_limit`を超えると、セッションは：

  1. 最後の`keep_last_n_turns`ターンの最も早いターンが開始される**前**のすべてを**要約**し、
  2. 保持される領域の先頭に**合成されたユーザー→アシスタントのペア**を挿入します：

     * `user`: `"Summarize the conversation we had so far."`（シャドウプロンプト）
     * `assistant`: `{生成された要約}`
  3. 最後の`keep_last_n_turns`ターンを**そのまま****保持**します。

これにより、最後の`keep_last_n_turns`ターンが発生した通りに正確に保持されることが保証され、それより前のすべてのコンテンツは2つの合成メッセージに圧縮されます。

In [291]:
session = SummarizingSession(
    keep_last_n_turns=2,
    context_limit=4,
    summarizer=LLMSummarizer(client)
)

In [292]:

# Example flow
await session.add_items([{"role": "user", "content": "Hi, my router won't connect. by the way, I am using Windows 10. I tried troubleshooting via your FAQs but I didn't get anywhere. This is my third tiem calling you. I am based in the US and one of Premium customers."}])
await session.add_items([{"role": "assistant", "content": "Let's check your firmware version."}])
await session.add_items([{"role": "user", "content": "Firmware v1.0.3; still failing."}])
await session.add_items([{"role": "assistant", "content": "Try a factory reset."}])
await session.add_items([{"role": "user", "content": "Reset done; error 42 now."}])
await session.add_items([{"role": "assistant", "content": "Try to install a new firmware."}])
await session.add_items([{"role": "user", "content": "I tried but I got another error now."}])
await session.add_items([{"role": "assistant", "content": "Can you please provide me with the error code?"}])
await session.add_items([{"role": "user", "content": "It says 404 not found when I try to access the page."}])
await session.add_items([{"role": "assistant", "content": "Are you connected to the internet?"}])
# At this point, with context_limit=4, everything *before* the earliest of the last 4 turns
# is summarized into a synthetic pair, and the last 2 turns remain verbatim.


In [293]:
history = await session.get_items()
# Pass `history` into your agent runner / responses call as the conversation context.

In [294]:
history

[{'role': 'user', 'content': 'Summarize the conversation we had so far.'},
 {'role': 'assistant',
  'content': '• Product & Environment:\n  - Router with Firmware v1.0.3, Windows 10, based in the US.\n\n• Reported Issue:\n  - Router fails to connect.\n\n• Steps Tried & Results:\n  - Checked FAQs: No resolution.\n  - Checked firmware version: v1.0.3, problem persists.\n  - Factory reset: Resulted in error 42.\n\n• Identifiers:\n  - Premium customer (no specific identifier provided).\n\n• Timeline Milestones:\n  - Initial troubleshooting via FAQs.\n  - Firmware check (before factory reset).\n  - Factory reset → Error 42.\n\n• Tool Performance Insights:\n  - Firmware version check successful.\n  - Factory reset resulted in new error (42).\n\n• Current Status & Blockers:\n  - Connection issue unresolved; error 42 is the immediate blocker.\n\n• Next Recommended Step:\n  - Install a new firmware update.'},
 {'role': 'user', 'content': 'I tried but I got another error now.'},
 {'role': 'assis

In [295]:
print(history[1]['content'])

• Product & Environment:
  - Router with Firmware v1.0.3, Windows 10, based in the US.

• Reported Issue:
  - Router fails to connect.

• Steps Tried & Results:
  - Checked FAQs: No resolution.
  - Checked firmware version: v1.0.3, problem persists.
  - Factory reset: Resulted in error 42.

• Identifiers:
  - Premium customer (no specific identifier provided).

• Timeline Milestones:
  - Initial troubleshooting via FAQs.
  - Firmware check (before factory reset).
  - Factory reset → Error 42.

• Tool Performance Insights:
  - Firmware version check successful.
  - Factory reset resulted in new error (42).

• Current Status & Blockers:
  - Connection issue unresolved; error 42 is the immediate blocker.

• Next Recommended Step:
  - Install a new firmware update.


`get_items_with_metadata`メソッドを使用して、デバッグや分析目的でメタデータを含むセッションの完全な履歴を取得できます。

In [ ]:
full_history = await session.get_items_with_metadata()

In [209]:
full_history

[{'message': {'role': 'user',
   'content': 'Summarize the conversation we had so far.'},
  'metadata': {'synthetic': True,
   'kind': 'history_summary_prompt',
   'summary_for_turns': '< all before idx 6 >'}},
 {'message': {'role': 'assistant',
   'content': '**Product & Environment:**\n- Device: Router\n- OS: Windows 10\n- Firmware: v1.0.3\n\n**Reported Issue:**\n- Router fails to connect to the internet, now showing error 42.\n\n**Steps Tried & Results:**\n- Checked FAQs: No resolution.\n- Firmware version checked: v1.0.3.\n- Factory reset performed: Resulted in error 42.\n\n**Identifiers:**\n- UNVERIFIED\n\n**Timeline Milestones:**\n- User attempted FAQ troubleshooting.\n- Firmware checked after initial advice.\n- Factory reset led to error 42.\n\n**Tool Performance Insights:**\n- FAQs and basic reset process did not resolve the issue.\n\n**Current Status & Blockers:**\n- Error 42 unresolved; firmware update needed.\n\n**Next Recommended Step:**\n- Install the latest firmware updat

In [210]:
print(history[1]['content'])

**Product & Environment:**
- Device: Router
- OS: Windows 10
- Firmware: v1.0.3

**Reported Issue:**
- Router fails to connect to the internet, now showing error 42.

**Steps Tried & Results:**
- Checked FAQs: No resolution.
- Firmware version checked: v1.0.3.
- Factory reset performed: Resulted in error 42.

**Identifiers:**
- UNVERIFIED

**Timeline Milestones:**
- User attempted FAQ troubleshooting.
- Firmware checked after initial advice.
- Factory reset led to error 42.

**Tool Performance Insights:**
- FAQs and basic reset process did not resolve the issue.

**Current Status & Blockers:**
- Error 42 unresolved; firmware update needed.

**Next Recommended Step:**
- Install the latest firmware update and check for resolution.


### 注意事項と設計上の選択

* **「新しい」側でターン境界を保持**: **`keep_last_n_turns`のユーザーターン**はそのまま保持され、それより古いものは圧縮されます。
* **2メッセージの要約ブロック**: 下流のツールが検出や表示を行いやすくなります（`metadata.synthetic == True`）。
* **非同期 + ロック規律**: （潜在的に遅い）要約処理の実行中は**ロックを解放**し、要約を適用する前に条件を再チェックして競合状態でのマージを回避します。
* **冪等な動作**: 要約処理中により多くのメッセージが到着した場合、await後の再チェックにより古い書き換えを防ぎます。

## Evals

最終的に、コンテキストエンジニアリングにおいても**evalsがすべて**です。重要な問いは次のとおりです：*モデルが「コンテキストを失っている」または「コンテキストを混同している」ことをどのように知ることができるでしょうか？*

メモリに関する完全なクックブックは将来的に独立したものになる可能性がありますが、まずは以下のような軽量な評価ハーネスのアイデアから始めることができます：

* **ベースライン & デルタ：** コアとなるevalセットを継続的に実行し、実験の前後を比較してメモリの改善を測定します。
* **LLM-as-Judge：** 慎重に設計された採点プロンプトを持つモデルを使用して、要約の品質を評価します。正しい形式で最も重要な詳細を捉えているかどうかに焦点を当てます。
* **トランスクリプト再生：** 長い会話を再実行し、コンテキストトリミングありとなしで次のターンの精度を測定します。メトリクスには、エンティティ/IDの完全一致や推論品質のルーブリックベースのスコアリングが含まれます。
* **エラー回帰追跡：** 一般的な失敗モード（未回答の質問、制約の脱落、不要/重複したツール呼び出し）を監視します。
* **トークン圧迫チェック：** トークン制限により保護されたコンテキストの削除が強制される場合にフラグを立てます。重要な詳細が削除されているタイミングを検出するために、前後のトークン数をログに記録します。

申し訳ございませんが、翻訳すべきテキストが提供されていないようです。「---」の後に翻訳したい英語のテキストを貼り付けていただけますでしょうか？

技術文書の翻訳を行う準備はできておりますので、翻訳したい内容をお送りください。